In [2]:
require "nyaplot"
require "nyaplot3d"
require "json"
require "daru"
require "statsample"

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\",\"THREE\":\"http://cdnjs.cloudflare.com/ajax/libs/three.js/r66/three.min\",\"Elegans\":\"https://cdn.rawgit.com/domitry/elegans/d81a728f62edaeeb67261516a272438fb39fa80a/release/elegans\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"},\"THREE\":{\"exports\":\"THREE\"},\"Elegans\":{\"exports\":\"Elegans\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');require(['THREE'], function(THREE){window['THREE']=THREE;console.log('finished loading THREE');require(['Elegans'], function(Eleg

true

In [4]:
sensor_names = {
  "32" => :ambient_temp,
  "96" => :systemboard_temp,
  "160" => :cpu1_temp,
  "224" => :cpu2_temp,
  "288" => :dimm_1a_temp,
  "1824" => :main_12v,
  "1888" => :main_5v,
  "1952" => :main_3_3v,
  "2272" => :cpu1_1_8v,
  "2336" => :cpu2_1_8v,
  "2400" => :fan1_sys,
  "2464" => :fan2_sys,
  "2528" => :fan3_sys,
  "2592" => :fan4_sys,
  "2656" => :fan5_sys,
  "3296" => :total_power
}

{"32"=>:ambient_temp, "96"=>:systemboard_temp, "160"=>:cpu1_temp, "224"=>:cpu2_temp, "288"=>:dimm_1a_temp, "1824"=>:main_12v, "1888"=>:main_5v, "1952"=>:main_3_3v, "2272"=>:cpu1_1_8v, "2336"=>:cpu2_1_8v, "2400"=>:fan1_sys, "2464"=>:fan2_sys, "2528"=>:fan3_sys, "2592"=>:fan4_sys, "2656"=>:fan5_sys, "3296"=>:total_power}

In [8]:
#response = JSON.load(File.read("6358153b-064b-5b11-bfc6-2a45cb0dc42a-24h.json"))
response = JSON.load(File.read("6358153b-064b-5b11-bfc6-2a45cb0dc42a-7d30m.json"))
data = response["result"]["data"][0]
time_idx = data["columns"].index { |i| i["name"] == "time" }
sensor_no_idx = data["columns"].index { |i| i["name"] == "sensor_no" }
value_idx = data["columns"].index { |i| i["name"] == "mean" }

h = sensor_names.values.each_with_object({time: []}) {|name, h| h[name] = []}
data["rows"].chunk_while { |i, j|
  i[time_idx] == j[time_idx]
}.each do |rows|
  h[:time].push(rows[0][time_idx])
  rows.each { |row|
    h[sensor_names[row[sensor_no_idx]]].push(row[value_idx])
  }
end
df = Daru::DataFrame.new(h, order: [:time] + sensor_names.values, dtype: :nmatrix)

,time,ambient_temp,systemboard_temp,cpu1_temp,cpu2_temp,dimm_1a_temp,main_12v,main_5v,main_3_3v,cpu1_1_8v,cpu2_1_8v,fan1_sys,fan2_sys,fan3_sys,fan4_sys,fan5_sys,total_power
0,1454918400000,292037.5,296175,305350,305850,295150,12000,5130,3400,1784.5,1780,6102000.0,6153000.0,5775000.0,6120000.0,6006000.0,92150
1,1454920200000,291724.0740740741,296150,305236.4197530864,305532.7160493827,295150,12000,5130,3400,1785.8024691358025,1780,6099259.259259259,6154074.074074074,5783703.703703703,6120000.0,6004444.444444444,92000
2,1454922000000,291278.0487804878,295491.46341463417,304430.48780487804,305369.51219512196,294174.39024390245,12000,5130,3400,1786.0975609756097,1780,6103902.439024391,6130243.902439024,5777560.975609756,6120000.0,6007317.073170732,92097.56097560975
3,1454923800000,290930.48780487804,295150,303967.0731707317,305040.243902439,294150,12000,5130,3400,1787.0731707317073,1780,6095121.951219512,6140487.804878049,5786341.463414635,6120000.0,6004390.243902439,92146.34146341463
4,1454925600000,290631.25,295137.5,303762.5,304587.5,293762.5,12000,5130,3400,1785.875,1780,6111000.0,6154500.0,5778000.0,6120000.0,6003000.0,93350
5,1454927400000,290569.7530864198,294668.51851851854,303779.6296296296,304557.4074074074,293150,12000,5130,3400,1788.3950617283951,1780,6102222.222222222,6134814.814814814,5779259.259259259,6120000.0,6000000.0,94518.51851851853
6,1454929200000,290406.25,294262.5,303425,304300,293150,12000,5130,3400,1788.375,1780,6102000.0,6124500.0,5779500.0,6120000.0,6003000.0,92000
7,1454931000000,290075.9259259259,294150,303224.0740740741,304310.4938271605,293150,12000,5130,3400,1787.4074074074074,1780,6105185.185185186,6122962.962962963,5785185.185185186,6120000.0,6013333.333333333,92049.38271604938
8,1454932800000,290046.34146341466,294150,303210.9756097561,304162.1951219512,293150,12000,5130,3400,1788.0487804878048,1780,6105365.853658536,6133170.731707317,5805365.853658536,6120000.0,6005853.658536585,92048.78048780488
9,1454934600000,289582.9268292683,293967.0731707317,303162.1951219512,303906.0975609756,292589.0243902439,12000,5130,3400,1788.780487804878,1780,6106829.268292683,6137560.975609756,5768780.4878048785,6120000.0,6002926.8292682925,92048.78048780488


In [9]:
df.describe

,ambient_temp,cpu1_1_8v,cpu1_temp,cpu2_1_8v,cpu2_temp,dimm_1a_temp,fan1_sys,fan2_sys,fan3_sys,fan4_sys,fan5_sys,main_12v,main_3_3v,main_5v,systemboard_temp,time,total_power
count,336,336,336,336,336,336,336,336,336,336,336,336,336,336,336,336,336
mean,291723.4267922329,1785.5751796405184,306002.4626994232,1780.0,305989.5566258756,294782.26308490144,6143914.268688747,6193032.706039078,5827531.923377082,6163384.803688227,6043446.333162838,11999.044665996722,3400.0,5130.0,295887.8166621238,1455219900000.0,92423.82928769314
std,3388.7922388812285,3.1015987460905783,2722.765225222841,0.0,2610.780687637149,3367.6906086024032,311700.7926433299,333754.7281917535,290311.3900885276,333260.9392342975,292373.19304556266,7.186675210921179,0.0,0.0,3433.24361815764,174850336.00196472,1557.1620614199442
min,283643.24324324325,1780,303150,1780,303150,286150,6083333.333333333,6121500.0,5766153.846153846,6120000.0,6000000.0,11940,3400,5130,287465.7894736842,1454918400000,92000
max,304136.4864864865,1790,317514.86486486485,1780,318150,306366.2162162162,9360000.0,9702162.162162162,8938378.378378378,9715135.135135135,9201081.081081081,12000,3400,5130,308136.4864864865,1455521400000,107891.8918918919


In [10]:
plot = Nyaplot::Plot.new
plot.add_with_df(df, :line, :time, :cpu1_temp)
plot.show

#<Nyaplot::Frame:0x007f91842b5a58 @properties={:panes=>[#<Nyaplot::Plot:0x007f91842b6598 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f91842b6390 @properties={:type=>:line, :options=>{:x=>:time, :y=>:cpu1_temp}, :data=>"9c1d391f-c0f4-4fd5-be6a-a4369e1de0b2"}, @xrange=[1454918400000, 1455521400000], @yrange=[303150, 317514.86486486485]>], :options=>{:zoom=>true, :width=>700, :xrange=>[1454918400000, 1455521400000], :yrange=>[303150, 317514.86486486485]}}>], :data=>{"9c1d391f-c0f4-4fd5-be6a-a4369e1de0b2"=>
#<Daru::DataFrame:70131481624320 @name = 9c1d391f-c0f4-4fd5-be6a-a4369e1de0b2 @size = 336>
                 time ambient_te systemboar  cpu1_temp  cpu2_temp dimm_1a_te   main_12v    main_5v  main_3_3v  cpu1_1_8v  cpu2_1_8v   fan1_sys   fan2_sys   fan3_sys   fan4_sys   fan5_sys total_powe 
         0 1454918400   292037.5     296175     305350     305850     295150      12000       5130       3400     1784.5       1780  6102000.0  6153000.0  5775000.0  6120000.0  6006000.0      92150 
         1 1454920200 291724.074     296150 305236.419 305532.716     295150      12000       5130       3400 1785.80246       1780 6099259.25 6154074.07 5783703.70  6120000.0 6004444.44      92000 
         2 1454922000 291278.048 295491.463 304430.487 305369.512 294174.390      12000       5130       3400 1786.09756       1780 6103902.43 6130243.90 5777560.97  6120000.0 6007317.07 92097.5609 
         3 1454923800 290930.487     295150 303967.073 305040.243     294150      12000       5130       3400 1787.07317       1780 6095121.95 6140487.80 5786341.46  6120000.0 6004390.24 92146.3414 
         4 1454925600  290631.25   295137.5   303762.5   304587.5   293762.5      12000       5130       3400   1785.875       1780  6111000.0  6154500.0  5778000.0  6120000.0  6003000.0      93350 
         5 1454927400 290569.753 294668.518 303779.629 304557.407     293150      12000       5130       3400 1788.39506       1780 6102222.22 6134814.81 5779259.25  6120000.0  6000000.0 94518.5185 
         6 1454929200  290406.25   294262.5     303425     304300     293150      12000       5130       3400   1788.375       1780  6102000.0  6124500.0  5779500.0  6120000.0  6003000.0      92000 
         7 1454931000 290075.925     294150 303224.074 304310.493     293150      12000       5130       3400 1787.40740       1780 6105185.18 6122962.96 5785185.18  6120000.0 6013333.33 92049.3827 
         8 1454932800 290046.341     294150 303210.975 304162.195     293150      12000       5130       3400 1788.04878       1780 6105365.85 6133170.73 5805365.85  6120000.0 6005853.65 92048.7804 
         9 1454934600 289582.926 293967.073 303162.195 303906.097 292589.024      12000       5130       3400 1788.78048       1780 6106829.26 6137560.97 5768780.48  6120000.0 6002926.82 92048.7804 
        10 1454936400  289068.75   293162.5     303150   303612.5     292150      12000       5130       3400   1788.625       1780  6108000.0  6127500.0  5784000.0  6120000.0  6010500.0      92050 
        11 1454938200     289025     293150     303150 303423.809     292150      12000       5130       3400 1788.80952       1780 6104285.71 6134285.71 5774285.71  6120000.0 6007142.85      92000 
        12 1454940000 289451.204 293403.012     303150 303752.409     292150      12000       5130       3400 1788.67469       1780 6105542.16 6128674.69 5780240.96  6120000.0 6010120.48      92000 
        13 1454941800 289553.614 294137.951     303150 304101.807 292595.783      12000       5130       3400 1787.71084       1780 6099759.03 6128674.69 5773012.04  6120000.0 6001445.78 92048.1927 
        14 1454943600 289497.560 293613.414 303162.195 303625.609     292150      12000       5130       3400 1789.02439       1780 6102439.02 6130243.90 5787804.87  6120000.0 6005853.65 92048.7804 
       ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ... 
}, :extensio

In [11]:
plot = Nyaplot::Plot.new
plot.add_with_df(df, :line, :time, :ambient_temp)
plot.show

#<Nyaplot::Frame:0x007f91849b4548 @properties={:panes=>[#<Nyaplot::Plot:0x007f91849b4f48 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f91849b4d90 @properties={:type=>:line, :options=>{:x=>:time, :y=>:ambient_temp}, :data=>"9c1d391f-c0f4-4fd5-be6a-a4369e1de0b2"}, @xrange=[1454918400000, 1455521400000], @yrange=[283643.24324324325, 304136.4864864865]>], :options=>{:zoom=>true, :width=>700, :xrange=>[1454918400000, 1455521400000], :yrange=>[283643.24324324325, 304136.4864864865]}}>], :data=>{"9c1d391f-c0f4-4fd5-be6a-a4369e1de0b2"=>
#<Daru::DataFrame:70131481624320 @name = 9c1d391f-c0f4-4fd5-be6a-a4369e1de0b2 @size = 336>
                 time ambient_te systemboar  cpu1_temp  cpu2_temp dimm_1a_te   main_12v    main_5v  main_3_3v  cpu1_1_8v  cpu2_1_8v   fan1_sys   fan2_sys   fan3_sys   fan4_sys   fan5_sys total_powe 
         0 1454918400   292037.5     296175     305350     305850     295150      12000       5130       3400     1784.5       1780  6102000.0  6153000.0  5775000.0  6120000.0  6006000.0      92150 
         1 1454920200 291724.074     296150 305236.419 305532.716     295150      12000       5130       3400 1785.80246       1780 6099259.25 6154074.07 5783703.70  6120000.0 6004444.44      92000 
         2 1454922000 291278.048 295491.463 304430.487 305369.512 294174.390      12000       5130       3400 1786.09756       1780 6103902.43 6130243.90 5777560.97  6120000.0 6007317.07 92097.5609 
         3 1454923800 290930.487     295150 303967.073 305040.243     294150      12000       5130       3400 1787.07317       1780 6095121.95 6140487.80 5786341.46  6120000.0 6004390.24 92146.3414 
         4 1454925600  290631.25   295137.5   303762.5   304587.5   293762.5      12000       5130       3400   1785.875       1780  6111000.0  6154500.0  5778000.0  6120000.0  6003000.0      93350 
         5 1454927400 290569.753 294668.518 303779.629 304557.407     293150      12000       5130       3400 1788.39506       1780 6102222.22 6134814.81 5779259.25  6120000.0  6000000.0 94518.5185 
         6 1454929200  290406.25   294262.5     303425     304300     293150      12000       5130       3400   1788.375       1780  6102000.0  6124500.0  5779500.0  6120000.0  6003000.0      92000 
         7 1454931000 290075.925     294150 303224.074 304310.493     293150      12000       5130       3400 1787.40740       1780 6105185.18 6122962.96 5785185.18  6120000.0 6013333.33 92049.3827 
         8 1454932800 290046.341     294150 303210.975 304162.195     293150      12000       5130       3400 1788.04878       1780 6105365.85 6133170.73 5805365.85  6120000.0 6005853.65 92048.7804 
         9 1454934600 289582.926 293967.073 303162.195 303906.097 292589.024      12000       5130       3400 1788.78048       1780 6106829.26 6137560.97 5768780.48  6120000.0 6002926.82 92048.7804 
        10 1454936400  289068.75   293162.5     303150   303612.5     292150      12000       5130       3400   1788.625       1780  6108000.0  6127500.0  5784000.0  6120000.0  6010500.0      92050 
        11 1454938200     289025     293150     303150 303423.809     292150      12000       5130       3400 1788.80952       1780 6104285.71 6134285.71 5774285.71  6120000.0 6007142.85      92000 
        12 1454940000 289451.204 293403.012     303150 303752.409     292150      12000       5130       3400 1788.67469       1780 6105542.16 6128674.69 5780240.96  6120000.0 6010120.48      92000 
        13 1454941800 289553.614 294137.951     303150 304101.807 292595.783      12000       5130       3400 1787.71084       1780 6099759.03 6128674.69 5773012.04  6120000.0 6001445.78 92048.1927 
        14 1454943600 289497.560 293613.414 303162.195 303625.609     292150      12000       5130       3400 1789.02439       1780 6102439.02 6130243.90 5787804.87  6120000.0 6005853.65 92048.7804 
       ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...

In [12]:
plot = Nyaplot::Plot.new
plot.add_with_df(df, :line, :time, :systemboard_temp)
plot.show

#<Nyaplot::Frame:0x007f9180d44a90 @properties={:panes=>[#<Nyaplot::Plot:0x007f9180d45490 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f9180d452d8 @properties={:type=>:line, :options=>{:x=>:time, :y=>:systemboard_temp}, :data=>"9c1d391f-c0f4-4fd5-be6a-a4369e1de0b2"}, @xrange=[1454918400000, 1455521400000], @yrange=[287465.7894736842, 308136.4864864865]>], :options=>{:zoom=>true, :width=>700, :xrange=>[1454918400000, 1455521400000], :yrange=>[287465.7894736842, 308136.4864864865]}}>], :data=>{"9c1d391f-c0f4-4fd5-be6a-a4369e1de0b2"=>
#<Daru::DataFrame:70131481624320 @name = 9c1d391f-c0f4-4fd5-be6a-a4369e1de0b2 @size = 336>
                 time ambient_te systemboar  cpu1_temp  cpu2_temp dimm_1a_te   main_12v    main_5v  main_3_3v  cpu1_1_8v  cpu2_1_8v   fan1_sys   fan2_sys   fan3_sys   fan4_sys   fan5_sys total_powe 
         0 1454918400   292037.5     296175     305350     305850     295150      12000       5130       3400     1784.5       1780  6102000.0  6153000.0  5775000.0  6120000.0  6006000.0      92150 
         1 1454920200 291724.074     296150 305236.419 305532.716     295150      12000       5130       3400 1785.80246       1780 6099259.25 6154074.07 5783703.70  6120000.0 6004444.44      92000 
         2 1454922000 291278.048 295491.463 304430.487 305369.512 294174.390      12000       5130       3400 1786.09756       1780 6103902.43 6130243.90 5777560.97  6120000.0 6007317.07 92097.5609 
         3 1454923800 290930.487     295150 303967.073 305040.243     294150      12000       5130       3400 1787.07317       1780 6095121.95 6140487.80 5786341.46  6120000.0 6004390.24 92146.3414 
         4 1454925600  290631.25   295137.5   303762.5   304587.5   293762.5      12000       5130       3400   1785.875       1780  6111000.0  6154500.0  5778000.0  6120000.0  6003000.0      93350 
         5 1454927400 290569.753 294668.518 303779.629 304557.407     293150      12000       5130       3400 1788.39506       1780 6102222.22 6134814.81 5779259.25  6120000.0  6000000.0 94518.5185 
         6 1454929200  290406.25   294262.5     303425     304300     293150      12000       5130       3400   1788.375       1780  6102000.0  6124500.0  5779500.0  6120000.0  6003000.0      92000 
         7 1454931000 290075.925     294150 303224.074 304310.493     293150      12000       5130       3400 1787.40740       1780 6105185.18 6122962.96 5785185.18  6120000.0 6013333.33 92049.3827 
         8 1454932800 290046.341     294150 303210.975 304162.195     293150      12000       5130       3400 1788.04878       1780 6105365.85 6133170.73 5805365.85  6120000.0 6005853.65 92048.7804 
         9 1454934600 289582.926 293967.073 303162.195 303906.097 292589.024      12000       5130       3400 1788.78048       1780 6106829.26 6137560.97 5768780.48  6120000.0 6002926.82 92048.7804 
        10 1454936400  289068.75   293162.5     303150   303612.5     292150      12000       5130       3400   1788.625       1780  6108000.0  6127500.0  5784000.0  6120000.0  6010500.0      92050 
        11 1454938200     289025     293150     303150 303423.809     292150      12000       5130       3400 1788.80952       1780 6104285.71 6134285.71 5774285.71  6120000.0 6007142.85      92000 
        12 1454940000 289451.204 293403.012     303150 303752.409     292150      12000       5130       3400 1788.67469       1780 6105542.16 6128674.69 5780240.96  6120000.0 6010120.48      92000 
        13 1454941800 289553.614 294137.951     303150 304101.807 292595.783      12000       5130       3400 1787.71084       1780 6099759.03 6128674.69 5773012.04  6120000.0 6001445.78 92048.1927 
        14 1454943600 289497.560 293613.414 303162.195 303625.609     292150      12000       5130       3400 1789.02439       1780 6102439.02 6130243.90 5787804.87  6120000.0 6005853.65 92048.7804 
       ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        .

In [13]:
plot = Nyaplot::Plot.new
plot.add_with_df(df, :scatter, :cpu1_temp, :systemboard_temp)
plot.show

#<Nyaplot::Frame:0x007f91846eca18 @properties={:panes=>[#<Nyaplot::Plot:0x007f91846ed418 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f91846ed260 @properties={:type=>:scatter, :options=>{:x=>:cpu1_temp, :y=>:systemboard_temp}, :data=>"9c1d391f-c0f4-4fd5-be6a-a4369e1de0b2"}, @xrange=[303150, 317514.86486486485], @yrange=[287465.7894736842, 308136.4864864865]>], :options=>{:zoom=>true, :width=>700, :xrange=>[303150, 317514.86486486485], :yrange=>[287465.7894736842, 308136.4864864865]}}>], :data=>{"9c1d391f-c0f4-4fd5-be6a-a4369e1de0b2"=>
#<Daru::DataFrame:70131481624320 @name = 9c1d391f-c0f4-4fd5-be6a-a4369e1de0b2 @size = 336>
                 time ambient_te systemboar  cpu1_temp  cpu2_temp dimm_1a_te   main_12v    main_5v  main_3_3v  cpu1_1_8v  cpu2_1_8v   fan1_sys   fan2_sys   fan3_sys   fan4_sys   fan5_sys total_powe 
         0 1454918400   292037.5     296175     305350     305850     295150      12000       5130       3400     1784.5       1780  6102000.0  6153000.0  5775000.0  6120000.0  6006000.0      92150 
         1 1454920200 291724.074     296150 305236.419 305532.716     295150      12000       5130       3400 1785.80246       1780 6099259.25 6154074.07 5783703.70  6120000.0 6004444.44      92000 
         2 1454922000 291278.048 295491.463 304430.487 305369.512 294174.390      12000       5130       3400 1786.09756       1780 6103902.43 6130243.90 5777560.97  6120000.0 6007317.07 92097.5609 
         3 1454923800 290930.487     295150 303967.073 305040.243     294150      12000       5130       3400 1787.07317       1780 6095121.95 6140487.80 5786341.46  6120000.0 6004390.24 92146.3414 
         4 1454925600  290631.25   295137.5   303762.5   304587.5   293762.5      12000       5130       3400   1785.875       1780  6111000.0  6154500.0  5778000.0  6120000.0  6003000.0      93350 
         5 1454927400 290569.753 294668.518 303779.629 304557.407     293150      12000       5130       3400 1788.39506       1780 6102222.22 6134814.81 5779259.25  6120000.0  6000000.0 94518.5185 
         6 1454929200  290406.25   294262.5     303425     304300     293150      12000       5130       3400   1788.375       1780  6102000.0  6124500.0  5779500.0  6120000.0  6003000.0      92000 
         7 1454931000 290075.925     294150 303224.074 304310.493     293150      12000       5130       3400 1787.40740       1780 6105185.18 6122962.96 5785185.18  6120000.0 6013333.33 92049.3827 
         8 1454932800 290046.341     294150 303210.975 304162.195     293150      12000       5130       3400 1788.04878       1780 6105365.85 6133170.73 5805365.85  6120000.0 6005853.65 92048.7804 
         9 1454934600 289582.926 293967.073 303162.195 303906.097 292589.024      12000       5130       3400 1788.78048       1780 6106829.26 6137560.97 5768780.48  6120000.0 6002926.82 92048.7804 
        10 1454936400  289068.75   293162.5     303150   303612.5     292150      12000       5130       3400   1788.625       1780  6108000.0  6127500.0  5784000.0  6120000.0  6010500.0      92050 
        11 1454938200     289025     293150     303150 303423.809     292150      12000       5130       3400 1788.80952       1780 6104285.71 6134285.71 5774285.71  6120000.0 6007142.85      92000 
        12 1454940000 289451.204 293403.012     303150 303752.409     292150      12000       5130       3400 1788.67469       1780 6105542.16 6128674.69 5780240.96  6120000.0 6010120.48      92000 
        13 1454941800 289553.614 294137.951     303150 304101.807 292595.783      12000       5130       3400 1787.71084       1780 6099759.03 6128674.69 5773012.04  6120000.0 6001445.78 92048.1927 
        14 1454943600 289497.560 293613.414 303162.195 303625.609     292150      12000       5130       3400 1789.02439       1780 6102439.02 6130243.90 5787804.87  6120000.0 6005853.65 92048.7804 
       ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...     

In [14]:
sr = Statsample::Regression.simple(df[:cpu1_temp], df[:systemboard_temp])
puts sr.summary

= Regression of cpu1_temp over systemboard_temp
  Table 1
+----------+------------+
| Variable |   Value    |
+----------+------------+
| r        | 0.920      |
| r^2      | 0.846      |
| a        | -58986.585 |
| b        | 1.160      |
| s.e      | 1349.827   |
+----------+------------+




In [15]:
cpu1_temp_min = df[:cpu1_temp].min
cpu1_temp_max = df[:cpu1_temp].max
df2 = Daru::DataFrame.new(
  cpu1_temp_min.step(cpu1_temp_max, (cpu1_temp_max - cpu1_temp_min) / 10.0).map { |x|
    { x: x, y: sr.a + sr.b * x }
  }
)
plot = Nyaplot::Plot.new  
plot.add_with_df(df, :scatter, :cpu1_temp, :systemboard_temp)
plot.add_with_df(df2, :line, :x, :y)
plot.show


#<Nyaplot::Frame:0x007f9184838980 @properties={:panes=>[#<Nyaplot::Plot:0x007f9184839d58 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f9184839ba0 @properties={:type=>:scatter, :options=>{:x=>:cpu1_temp, :y=>:systemboard_temp}, :data=>"9c1d391f-c0f4-4fd5-be6a-a4369e1de0b2"}, @xrange=[303150, 317514.86486486485], @yrange=[287465.7894736842, 308136.4864864865]>, #<Nyaplot::Diagram:0x007f91848391f0 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"fa0d542e-f0b1-4f2d-8eb5-3ce70bec0a4c"}, @xrange=[303150.0, 317514.86486486485], @yrange=[292579.784480806, 309238.8753298689]>], :options=>{:zoom=>true, :width=>700, :xrange=>[303150, 317514.86486486485], :yrange=>[287465.7894736842, 309238.8753298689]}}>], :data=>{"9c1d391f-c0f4-4fd5-be6a-a4369e1de0b2"=>
#<Daru::DataFrame:70131481624320 @name = 9c1d391f-c0f4-4fd5-be6a-a4369e1de0b2 @size = 336>
                 time ambient_te systemboar  cpu1_temp  cpu2_temp dimm_1a_te   main_12v    main_5v  main_3_3v  cpu1_1_8v  cpu2_1_8v   fan1_sys   fan2_sys   fan3_sys   fan4_sys   fan5_sys total_powe 
         0 1454918400   292037.5     296175     305350     305850     295150      12000       5130       3400     1784.5       1780  6102000.0  6153000.0  5775000.0  6120000.0  6006000.0      92150 
         1 1454920200 291724.074     296150 305236.419 305532.716     295150      12000       5130       3400 1785.80246       1780 6099259.25 6154074.07 5783703.70  6120000.0 6004444.44      92000 
         2 1454922000 291278.048 295491.463 304430.487 305369.512 294174.390      12000       5130       3400 1786.09756       1780 6103902.43 6130243.90 5777560.97  6120000.0 6007317.07 92097.5609 
         3 1454923800 290930.487     295150 303967.073 305040.243     294150      12000       5130       3400 1787.07317       1780 6095121.95 6140487.80 5786341.46  6120000.0 6004390.24 92146.3414 
         4 1454925600  290631.25   295137.5   303762.5   304587.5   293762.5      12000       5130       3400   1785.875       1780  6111000.0  6154500.0  5778000.0  6120000.0  6003000.0      93350 
         5 1454927400 290569.753 294668.518 303779.629 304557.407     293150      12000       5130       3400 1788.39506       1780 6102222.22 6134814.81 5779259.25  6120000.0  6000000.0 94518.5185 
         6 1454929200  290406.25   294262.5     303425     304300     293150      12000       5130       3400   1788.375       1780  6102000.0  6124500.0  5779500.0  6120000.0  6003000.0      92000 
         7 1454931000 290075.925     294150 303224.074 304310.493     293150      12000       5130       3400 1787.40740       1780 6105185.18 6122962.96 5785185.18  6120000.0 6013333.33 92049.3827 
         8 1454932800 290046.341     294150 303210.975 304162.195     293150      12000       5130       3400 1788.04878       1780 6105365.85 6133170.73 5805365.85  6120000.0 6005853.65 92048.7804 
         9 1454934600 289582.926 293967.073 303162.195 303906.097 292589.024      12000       5130       3400 1788.78048       1780 6106829.26 6137560.97 5768780.48  6120000.0 6002926.82 92048.7804 
        10 1454936400  289068.75   293162.5     303150   303612.5     292150      12000       5130       3400   1788.625       1780  6108000.0  6127500.0  5784000.0  6120000.0  6010500.0      92050 
        11 1454938200     289025     293150     303150 303423.809     292150      12000       5130       3400 1788.80952       1780 6104285.71 6134285.71 5774285.71  6120000.0 6007142.85      92000 
        12 1454940000 289451.204 293403.012     303150 303752.409     292150      12000       5130       3400 1788.67469       1780 6105542.16 6128674.69 5780240.96  6120000.0 6010120.48      92000 
        13 1454941800 289553.614 294137.951     303150 304101.807 292595.783      12000       5130       3400 1787.71084       1780 6099759.03 6128674.69 5773012.04  6120000.0 6001445.78 92048.1927 
        14 1454943600 289497.560 293613.414 303162.195 303625.609     292150      12000       5130       3400 1789.02439       1780 6102439.02 6130243.9

In [49]:
open("correlation_matrix.csv", "w") do |f|
  columns = df.vectors.to_a.map(&:to_s)
  f.puts("," + columns.join(","))
  Statsample::Bivariate.correlation_matrix(df).rows.each_with_index do |row, i|
    f.puts(columns[i] + "," + row.join(","))
  end
end

[[1.0, 0.11556339584553517, 0.12401486019457736, 0.24702517504153823, 0.1668372701661, 0.13980834450074422, -0.023039343852875423, NaN, NaN, -0.18420521035408116, NaN, 0.020752697200011373, 0.042299098437928354, 0.025767684865030675, 0.02286166977591128, 0.02302269617200265, 0.01951333164311545], [0.11556339584553517, 1.0, 0.9950900318718736, 0.9185751977823455, 0.9298901571440615, 0.9965801707902164, -0.4232872895248966, NaN, NaN, -0.9062044961742884, NaN, 0.408889831943291, 0.4392719241687859, 0.4118580968560804, 0.4220835600067833, 0.41273484043632197, 0.35692349715213967], [0.12401486019457736, 0.9950900318718736, 1.0, 0.91971935393018, 0.9317770222528541, 0.9958379746753179, -0.41190815230411204, NaN, NaN, -0.9218420579757527, NaN, 0.3968863413387115, 0.4296375048837056, 0.40060892818971, 0.4104863067274483, 0.4010879385412933, 0.34622251931686543], [0.24702517504153823, 0.9185751977823455, 0.91971935393018, 1.0, 0.9824571914788347, 0.9182336619305786, -0.48747813123359685, NaN, N